In [1]:
using Revise

In [2]:
using JLD2
using FileIO
import LearningMPC

In [3]:
addprocs(10)

10-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11

In [4]:
@everywhere using Flux
@everywhere using MLDataPattern
@everywhere using CoordinateTransformations
@everywhere using ProgressMeter
@everywhere import FluxExtensions

In [5]:
samples = load("../2018-02-07-hopper-smaller-grid/grid_search.jld2")["samples"];

In [6]:
filter!(samples) do sample
    sample.state[2] <= sample.state[1] && !any(isnan, sample.uJ)
end;
length(samples)

59931

In [7]:
features(sample::LearningMPC.Sample) = (sample.state, sample.uJ[:, 1], sample.uJ[:, 2:end])
data = features.(samples);
train_data, test_data = splitobs(shuffleobs(data), at=0.85);

In [14]:
@everywhere function attention_model()
    signals = Chain(Dense(4, 16))
    weights = Chain(
        LinearMap(UniformScaling(0.2)),
        Dense(4, 16, elu),
        Dense(16, 16, elu),
        softmax)
    model = FluxExtensions.Attention(signals, weights)
    @assert size(model(zeros(4))) == (1,)
            
    loss = (x, u, J) -> Flux.mse(model(x), u[2])
    model, loss
end

@everywhere function layered_model()
    model = Chain(
        LinearMap(UniformScaling(0.2)),
        Dense(4, 16, elu),
        Dense(16, 16, elu),
        Dense(16, 1))
    
    @assert size(model(zeros(4))) == (1,)
    loss = (x, u, J) -> Flux.mse(model(x), u[2])
    model, loss
end

@everywhere function attention_model_tangent(weight=0.05)
    signals = Chain(Dense(4, 16))
    weights = Chain(
        LinearMap(UniformScaling(0.2)),
        Dense(4, 16, elu),
        Dense(16, 16, elu),
        softmax,
    )
    model = FluxExtensions.TangentPropagator(FluxExtensions.Attention(signals, weights))
            
    @assert size.(model(zeros(4))) == ((1,), (1, 4))
    
    function loss(x, u, J)
        uhat, Jhat = model(x)
        (1 - weight) * Flux.mse(uhat, u[2]) + weight * Flux.mse(Jhat, J[2, :])
    end
    function test_loss(x, u, J)
        uhat, Jhat = model(x)
        Flux.mse(uhat, u[2])
    end
    model, loss, test_loss
end

@everywhere function layered_model_tangent(weight=0.05)
    model = FluxExtensions.TangentPropagator(Chain(
        LinearMap(UniformScaling(0.2)),
        Dense(4, 16, elu),
        Dense(16, 16, elu),
        Dense(16, 1)))
            
    @assert size.(model(zeros(4))) == ((1,), (1, 4))
    
    function loss(x, u, J)
        uhat, Jhat = model(x)
        (1 - weight) * Flux.mse(uhat, u[2]) + weight * Flux.mse(Jhat, J[2, :])
    end
    function test_loss(x, u, J)
        uhat, Jhat = model(x)
        Flux.mse(uhat, u[2])
    end
    model, loss, test_loss
end

In [ ]:
iterations = 100

attention_losses = @parallel (vcat) for (train, validation) in collect(kfolds(train_data; k=10))
    model, loss = attention_model()
    opt = Flux.ADADelta(params(model))
    (model, map(1:iterations) do i
        Flux.train!(loss, train, opt)
        (mean(xy -> Flux.Tracker.value(loss(xy...)), validation),)
    end)
end
@show attention_losses

layered_losses = @parallel (vcat) for (train, validation) in collect(kfolds(train_data; k=10))
    model, loss = layered_model()
    opt = Flux.ADADelta(params(model))
    (model, map(1:iterations) do i
        Flux.train!(loss, train, opt)
        (mean(xy -> Flux.Tracker.value(loss(xy...)), validation),)
    end)
end
@show layered_losses

attention_losses_tangent = @parallel (vcat) for (train, validation) in collect(kfolds(train_data; k=10))
    model, loss, test_loss = attention_model_tangent()
    opt = Flux.ADADelta(params(model))
    (model, map(1:iterations) do i
        Flux.train!(loss, train, opt)
        (mean(xy -> Flux.Tracker.value(loss(xy...)), validation), mean(xy -> Flux.Tracker.value(test_loss(xy...)), validation))
    end)
end
@show attention_losses_tangent

layered_losses_tangent = @parallel (vcat) for (train, validation) in collect(kfolds(train_data; k=10))
    model, loss, test_loss = layered_model_tangent()
    opt = Flux.ADADelta(params(model))
    (model, map(1:iterations) do i
        Flux.train!(loss, train, opt)
        (mean(xy -> Flux.Tracker.value(loss(xy...)), validation), mean(xy -> Flux.Tracker.value(test_loss(xy...)), validation))
    end)
end
@show layered_losses_tangent



In [17]:
save("losses2.jld2", "attention", attention_losses, "layered", layered_losses, "attention_tangent", attention_losses_tangent, "layered_tangent", layered_losses_tangent)

In [19]:
using Plots; gr()

Plots.GRBackend()

In [22]:
typeof(attention_losses[1])

FluxExtensions.Attention{Flux.Chain,Flux.Chain}

In [29]:
plt = plot(legend=nothing)
for losses in attention_losses[2:2:end]
    plot!(plt, last.(losses), linecolor="red")
end
for losses in layered_losses[2:2:end]
    plot!(plt, last.(losses), linecolor="blue")
end
for losses in attention_losses_tangent[2:2:end]
    plot!(plt, last.(losses), linecolor="pink")
end
for losses in layered_losses_tangent[2:2:end]
    plot!(plt, last.(losses), linecolor="skyblue")
end
plt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 125 
 
 
 150 
 
 
 175 
 
<polyline clip-path="url(#clip2602)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 35.5169,43.8116 41.0199,72.4727 46.5229,109.972 52.026,155.341 57.529,204.416 63.032,241.319 68.535,269.548 74.038,291.339 79.541,308.639 85.044,322.195 
 90.547,332.218 96.05,339.277 101.553,344.232 107.056,347.838 112.559,350.604 118.062,352.811 123.565,354.601 129.068,356.027 134.571,357.13 140.074,357.956 
 145.577,358.549 151.08,358.883 156.583,358.964 162.086,358.956 167.589,358.922 173.092,358.862 178.595,358.801 184.098,358.759 189.601,358.746 195.104,358.753 
 200.607,358.789 206.11,358.942 211.613,359.094 217.116,359.357 222.619,359.733 228.122,360.013 233.625,360.214 239.128,360.408 244.631,360.619 250.134,360.855 
 255.637,361.122 261.14,361.407 266.643,361.704 272.146,362.011 277.649,362.314 283.152,362.667 288.655,363.363 294.158,364.192 299.661,364.705 305.164,365.112 
 310.667,365.521 316.17,365.898 321.673,366.233 327.176,366.524 332.679,366.767 338.182,366.959 343.686,367.102 349.189,367.209 354.692,367.294 360.195,367.36 
 365.698,367.415 371.201,367.452 376.704,367.486 382.207,367.641 387.71,368.101 393.213,368.639 398.716,369.089 404.219,369.467 409.722,369.814 415.225,370.159 
 420.728,370.508 426.231,370.851 431.734,371.183 437.237,371.501 442.74,371.807 448.243,372.103 453.746,372.393 459.249,372.685 464.752,372.978 470.255,373.268 
 475.758,373.554 481.261,373.828 486.764,374.084 492.267,374.322 497.77,374.547 503.273,374.766 508.776,374.983 514.279,375.192 519.782,375.378 525.285,375.529 
 530.788,375.64 536.291,375.719 541.794,375.772 547.297,375.801 552.8,375.813 558.303,375.813 563.806,375.805 569.309,375.792 574.812,375.774 580.315,375.752 
 
 "/>
<polyline clip-path="url(#clip2602)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 35.5169,27.7998 41.0199,57.6793 46.5229,109.39 52.026,163.056 57.529,208.918 63.032,246.997 68.535,276.224 74.038,298.71 79.541,315.974 85.044,328.84 
 90.547,337.988 96.05,344.28 101.553,348.622 107.056,351.706 112.559,353.981 118.062,355.714 123.565,357.061 129.068,358.119 134.571,358.945 140.074,359.572 
 145.577,360.028 151.08,360.355 156.583,360.587 162.086,360.75 167.589,360.864 173.092,360.946 178.595,361.006 184.098,361.051 189.601,361.087 195.104,361.114 
 200.607,361.136 206.11,361.154 211.613,361.167 217.116,361.176 222.619,361.182 228.122,361.184 233.625,361.184 239.128,361.18 244.631,361.173 250.134,361.165 
 255.637,361.155 261.14,361.144 266.643,361.132 272.146,361.12 277.649,361.108 283.152,361.069 288.655,361.023 294.158,360.988 299.661,360.961 305.164,360.931 
 310.667,360.904 316.17,360.883 321.673,360.866 327.176,360.852 332.679,360.84 338.182,360.831 343.686,360.825 349.189,360.82 354.692,360.818 360.195,360.816 
 365.698,360.816 371.201,360.817 376.704,360.82 382.207,360.822 387.71,360.825 393.213,360.829 398.716,360.833 404.219,360.836 409.722,360.84 415.225,360.845 
 420.728,360.85 426.231,360.855 431.734,360.861 437.237,360.868 442.74,360.875 448.243,360.883 453.746,360.892 459.249,360.902 464.752,360.914 470.255,360.931 
 475.758,360.959 481.261,361.008 486.764,361.076 492.267,361.147 497.77,361.212 503.273,361.261 508.776,361.328 514.279,361.386 519.782,361.44 525.285,361.5 
 530.788,361.574 536.291,361.666 541.794,361.773 547.297,361.892 552.8,362.02 558.303,362.154 563.806,362.291 569.309,362.432 574.812,362.577 580.315,362.734 
 
 "/>
<polyline clip-path="url(#clip2602)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 35.5169,33.928 41.0199,56.9893 46.5229,79.885 52.026,102.337 57.529,123.127 63.032,144.773 68.535,189.488 74.038,230.535 79.541,259.537 85.044,282.832 
 90.547,301.483 96.05,316.344 101.553,327.979 107.056,336.784 112.55